# 2023-12-01 What is uncertainty (also called error or precision) in science?

## Overview

Any scientific result should be presented as a measured value and estimated uncertainty in that value. There are multiple ways to think about the uncertainty, depending on the circumstances. 

It is almost never correct to think about it as an "error" -- that implies it is something you can fix if you just try harder. If that is the case you should really just try harder before reporting the result.

There are two fairly common cases that are useful to think about:

+ You are measuring a process that is truly random. No matter how good your equipment is you will measure a diffferent value every time. In this case it is useful to think about the uncertainty as expressing the range of values you should expect to see if you repeat the experiment. An example of this is measuring the flux from a star: it is a random variable and you would measure a different value each time even if circumstances were identical.
+ There is a limit on the precision of what you can measure. In this case, future measurements might be better than what you can do now. In this case it is useful to think about the uncertainty as expression the range of values consistent with the precision with which you can measure that thing. An example of this from astronomy is time: we can measure time much more precisely now than we could 100 years ago.

### One goal for today: get a sense of uncertainty in measuring exoplanet properties

## Run the wall of code below -- it is a transit measurement for TIC 402828941

In [ ]:
%matplotlib widget

from astropy.table import Table 
from astropy.time import Time
from astropy.timeseries import TimeSeries, aggregate_downsample
from astropy import units as u

import ipywidgets as ipw
import mpl_interactions.ipyplot as iplt
import matplotlib.pyplot as plt
import numpy as np
import batman


In [ ]:
tab = Table.read('TIC-402828941-2022-08-04-transformed-relative-flux.csv')
tic = tab[tab['star_id'] == 1]
#tic.write('TIC-402828941-2022-08-04-transformed-relative-flux.csv', overwrite=True)
t_mean = tic['BJD'].mean()

t_ob = Time(tic['BJD'], scale='tdb', format='jd')

In [ ]:
bad_beginning = t_ob < t_ob[0] + 15 * u.min

tic = tic[~bad_beginning]
t_ob = t_ob[~bad_beginning]

after_transit = t_ob.jd > 2459795.85

norm_fac = 1 / tic['relative_flux'][after_transit].mean()


tic['relative_flux'] *= norm_fac

In [ ]:

ts = TimeSeries([tic['relative_flux']], time=t_ob)

bts = aggregate_downsample(ts, time_bin_size=10 * u.min)

In [ ]:
params = batman.TransitParams()

params.t0 = t_mean                        #time of inferior conjunction
params.per = 1.                       #orbital period
params.rp = 0.1                       #planet radius (in units of stellar radii)
params.a = 15.                        #semi-major axis (in units of stellar radii)
params.inc = 87.                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 90.                        #longitude of periastron (in degrees)
params.limb_dark = "quadratic"        #limb darkening model
params.u = [0.1, 0.1]      #limb darkening coefficients [u1, u2, u3, u4]

t = t_mean + np.linspace(-0.2, 0.2, 1000)  #times at which to calculate light curve
m = batman.TransitModel(params, t)    #initializes model

In [ ]:
fit_by_peeps = Table.read('exoplanet-params.csv')
print(fit_by_peeps.colnames)
n_plots = len(fit_by_peeps)

In [ ]:
n_cols = n_plots // 2
n_rows = n_plots // n_cols

In [ ]:
fig = plt.figure(layout="constrained")
ax_array = fig.subplots(n_cols, n_rows, sharex=True, sharey=True)

num = 0
for row, ax in zip(fit_by_peeps, ax_array.flatten()):
    ax.plot(tic['BJD'], tic['relative_flux'], '.', color='red', alpha=0.2)
    ax.plot(bts.time_bin_center.jd, bts['relative_flux'], '.', color='green')
    params.rp = row['Rp/R*']
    params.a = row['a/R*']
    params.t0 = row['T_c']
    ax.plot(t, m.light_curve(params))
    ax.legend(f'{num}')
    num += 1
    ax.grid()

In [ ]:
fig = plt.figure(layout="constrained")
ax_array = fig.subplots(n_cols, n_rows, sharex=True, sharey=True, )

scores = {}

num = 0

for row, ax in zip(fit_by_peeps, ax_array.flatten()):
    # ax.plot(tic['BJD'], tic['relative_flux'], '.', color='red', alpha=0.2)
    # ax.plot(bts.time_bin_center.jd, bts['relative_flux'], '.', color='green')
    params.rp = row['Rp/R*']
    params.a = row['a/R*']
    params.t0 = row['T_c']
    m2 = batman.TransitModel(params, tic['BJD'])
    residual = tic['relative_flux'] - m2.light_curve(params)
    rms = np.sqrt((residual**2).sum())
    cen = residual.mean()
    scores[row['Name']] = rms
    ax.plot(tic['BJD'], residual, '.', color='red', label=f'{rms=:.3f}, {cen=:.3f}', alpha=0.2)
    ax.plot([tic['BJD'].min(), tic['BJD'].max()], [0, 0], color='black')
    #ax.legend()
    ax.set_title(f'{num} {rms=:.3f}, {cen=:.3f}')
    num += 1
#    ax.plot(t, m.light_curve(params))
    ax.grid()
    ax.set_ylim(-0.03, 0.03)
    

In [ ]:
dot_plot_x = fit_by_peeps['Rp/R*']
dot_plot_y = fit_by_peeps['a/R*']
color = np.array(list(scores.values()))
plt.ylim(2.5, 5.5)
plt.xlim(0.06, 0.17)
plt.figure()
plt.scatter(dot_plot_x, dot_plot_y, c=color)
plt.ylim(2.5, 5.5)
plt.xlim(0.06, 0.17)
plt.xlabel('Rp/R*')
plt.ylabel('a/R*')
plt.grid()